# Option 1: Heroes of Pymoli

---

In [1]:
# dependencies
import pandas as pd
import numpy as np

# data file name
filename = "Resources/purchase_data.csv"

# read data file and load it into a data frame
df = pd.read_csv(filename)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* *Display Total Number of Players*. Each player has a unique SN (Screen Name), so if we count unique Screen Names, we count unique players as well.

In [2]:
pd.DataFrame([{'Total Players': df['SN'].nunique()}])

,Total Players
0,576


## Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue.

In [3]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

pd.DataFrame([{'Number of Unique Items': df['Item ID'].nunique(),
               'Average Price':          df['Price'].mean(),
               'Number of Purchases'   : df['Purchase ID'].nunique(),
               'Total Revenue'         : df['Price'].sum()}],
             columns = ['Number of Unique Items',
                        'Average Price',
                        'Number of Purchases',
                        'Total Revenue'])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [4]:
# this is to format the floats as percentage amounts with two decimals
pd.options.display.float_format = '{:,.2f}%'.format

# strategy:
# 1) get a dataframe listing all SNs (Screen Names) and their reported genders,
# 2) delete duplicates from resulting dataframe,
# 3) do a groupby gender in previous dataframe, and
# 4) for the percentage calculation use a lambda function to calculate 
#    proportion of counts by gender against total count.
unique_players_df = df[['SN', 'Gender']].drop_duplicates()
players_by_gender_df = unique_players_df.groupby(['Gender']).count()
players_by_gender_df.rename(columns={'SN': 'Total Count'}, inplace=True)
players_by_gender_df['Percentage of Players'] = players_by_gender_df.apply(lambda x: 100 * x / x.sum())
players_by_gender_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [5]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

# strategy:
# 1) get a groupby on two columns of original dataframe: Gender and Price,
# 2) calculate count, Price average, and Price sum using aggregation functions,
# 3) for average purchase per user, get number of users by gender from 
#    dataframe created in previous section (players_by_gender_df) thru a merge, and
# 4) drop unneeded columns now calculation of last variable is over.
gender_groupby = df[['Gender', 'Price']].groupby(['Gender'])
purchases_by_gender_df = gender_groupby.count()
purchases_by_gender_df.rename(columns={'Price': 'Purchase Count'}, inplace=True)
purchases_by_gender_df['Average Purchase Price'] = gender_groupby.mean()
purchases_by_gender_df['Total Purchase Value'] = gender_groupby.sum()
purchases_by_gender_df = pd.merge(players_by_gender_df, purchases_by_gender_df, how='inner', on='Gender')
purchases_by_gender_df['Avg Total Purchase per Person'] = purchases_by_gender_df['Total Purchase Value']/purchases_by_gender_df['Total Count']
purchases_by_gender_df.drop(columns=['Total Count', 'Percentage of Players'], inplace=True)
purchases_by_gender_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40+)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

In [6]:
# this is to format the floats as percentage amounts with two decimals
pd.options.display.float_format = '{:,.2f}%'.format

# strategy:
# 1) get a dataframe listing all SNs (Screen Names) and their reported ages,
# 2) delete duplicates from resulting dataframe,
# 3) cut data in bins based on age groups using pd.cut,
# 4) get a counts per bin using value_counts(), and
# 5) for the percentage calculation use a lambda function to calculate 
#    proportion of counts by age group against total count.
unique_players_df = df[['SN', 'Age']].drop_duplicates()
ages_in_bins_df = pd.DataFrame(pd.cut(unique_players_df['Age'],
                               [0, 10, 15, 20, 25, 30, 35, 40, 200],
                               labels = ['0-10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'],
                               right=False))
players_by_age_group_df = ages_in_bins_df['Age'].value_counts().to_frame(name='Total Count')
players_by_age_group_df.sort_index(inplace=True)
players_by_age_group_df.rename(index={'0-10': '<10'}, inplace=True)
players_by_age_group_df['Percentage of Players'] = players_by_age_group_df.apply(lambda x: 100 * x / x.sum())
players_by_age_group_df.index.names = ['Age']
players_by_age_group_df

,Total Count,Percentage of Players
Age,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Run basic calculations to obtain:
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Total Purchase per Person

In [7]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

# strategy:
# 1) create a dataframe using original dataframe columns 'Age', and 'Price',
# 2) cut data in bins based on age groups using pd.cut and add bins to
#    dataframe from step 1),
# 3) create groupby based on 'Age' where 'Age' is coming from the bins in step 2),
# 4) calculate count, Price average, and Price sum using aggregation functions,
# 5) for average purchase per user, get number of users by age group from 
#    dataframe created in previous section (players_by_age_group_df) thru a merge, and
# 6) drop unneeded columns now calculation of last variable is over.
purchases_by_age_df = df.loc[:, ['Age', 'Price']]
purchases_by_age_df['Age Group'] = pd.cut(purchases_by_age_df['Age'],
                                          [0, 10, 15, 20, 25, 30, 35, 40, 200],
                                          labels = ['0-10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'],
                                          right=False)
purchases_by_age_df.set_index('Age Group', inplace=True)
purchases_by_age_df.drop(columns=['Age'], inplace=True)
purchases_by_age_df.index.names = ['Age']
age_group_groupby = purchases_by_age_df.groupby('Age')
purchases_by_age_group_df = age_group_groupby.count()
purchases_by_age_group_df.rename(columns={'Price': 'Purchase Count'}, inplace=True)
purchases_by_age_group_df['Average Purchase Price'] = age_group_groupby.mean()
purchases_by_age_group_df['Total Purchase Value'] = age_group_groupby.sum()
purchases_by_age_group_df.rename(index={'0-10': '<10'}, inplace=True)
purchases_by_age_group_df = pd.merge(players_by_age_group_df, purchases_by_age_group_df, how='inner', on='Age')
purchases_by_age_group_df['Avg Total Purchase per Person'] = purchases_by_age_group_df['Total Purchase Value']/purchases_by_age_group_df['Total Count']
purchases_by_age_group_df.drop(columns=['Total Count', 'Percentage of Players'], inplace=True)
purchases_by_age_group_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
* SN
* Purchase Count
* Average Purchase Price
* Total Purchase Value

In [8]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

# strategy:
# 1) create a dataframe using original dataframe columns 'SN', and 'Price',
# 2) do a groupby 'SN',
# 3) calculate count, average, and sum using aggregation functions on groupby, and
# 4) sort resulting dataframe by total, and show first five with dataframe head function.
purchases_by_user_df = df[['SN', 'Price']]
by_user_groupby = purchases_by_user_df.groupby('SN')
spenders = by_user_groupby.count()
spenders.rename(columns={'Price': 'Purchase Count'}, inplace=True)
spenders['Average Purchase Price'] = by_user_groupby.mean()
spenders['Total Purchase Value'] = by_user_groupby.sum()
spenders = spenders.sort_values('Total Purchase Value', ascending=False)
spenders.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

Identify the 5 most popular items by purchase count, then list (in a table):
* Item ID
* Item Name
* Purchase Count
* Item Price
* Total Purchase Value

In [9]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

# strategy:
# 1) create a dataframe using original dataframe columns 'Item ID', 'Item Name' and 'Price',
# 2) do a groupby ['Item ID', 'Item Name'],
# 3) calculate count, average, and sum using aggregation functions on groupby, and
# 4) sort resulting dataframe by count, and show first five with dataframe head function.
purchases_by_item_df = df[['Item ID', 'Item Name', 'Price']]
by_item_groupby = purchases_by_item_df.groupby(['Item ID', 'Item Name'])
items = by_item_groupby.count()
items.rename(columns={'Price': 'Purchase Count'}, inplace=True)
items['Item Price'] = by_item_groupby.mean()
items['Total Purchase Value'] = by_item_groupby.sum()
items = items.sort_values('Purchase Count', ascending=False)
items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

Identify the 5 most profitable items by total purchase value, then list (in a table):
* Item ID
* Item Name
* Purchase Count
* Item Price
* Total Purchase Value

In [10]:
# this is to format the floats as dollar amounts with two decimals
pd.options.display.float_format = '${:,.2f}'.format

# strategy:
# 1) create a dataframe using original dataframe columns 'Item ID', 'Item Name' and 'Price',
# 2) do a groupby ['Item ID', 'Item Name'],
# 3) calculate count, average, and sum using aggregation functions on groupby, and
# 4) sort resulting dataframe by total, and show first five with dataframe head function.
purchases_by_item_df = df[['Item ID', 'Item Name', 'Price']]
by_item_groupby = purchases_by_item_df.groupby(['Item ID', 'Item Name'])
items = by_item_groupby.count()
items.rename(columns={'Price': 'Purchase Count'}, inplace=True)
items['Item Price'] = by_item_groupby.mean()
items['Total Purchase Value'] = by_item_groupby.sum()
items = items.sort_values('Total Purchase Value', ascending=False)
items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


---

## Observable Trends (two from the sample + three of my own)

1. Of the 576 active players, the vast majority are male (84%).
2. Most of the players are between 20 and 24 years old.
3. The age group that spends the most in average is between 35 and 39 years old.
4. Top item purchased, and the most profitable as well, is Item ID 178 (Oathbreaker, Last Hope of the Breaking Storm).
5. User who spent the most was Lisosia93.